In [1]:
import pandas as pd
import numpy as np
import os
import requests
from io import BytesIO
import json
import yadisk
from datetime import datetime, date, timedelta
import re
import gspread
from oauth2client.service_account import ServiceAccountCredentials

import config
from yandex_disk_func import *
from parse_functions import *
from google_connector import *

# # указываем путь и название файла с токеном для Яндекс Диск
file_path = r'C:\Users\o.bogomolov\Desktop\Jupyter_notebook\35_mediaplan_parser'
# забираем токен для подключения к гугл
service_key = config.service
google_sheet_link = 'https://docs.google.com/spreadsheets/d/11hF_txYEbKrQImRQ0x3Fia3PyJvDAqtpGI25idV7oa4/edit?usp=sharing'
gmail = config.gmail
sheet_name = 'prog'

# обычная ссылка на доступ к папке одного данного ФЛАЙТА из личного кабинета
public_key = config.public_key 

# забираем список папок в нужной нам директории
res = get_yandex_disk_folders(public_key)
yandex_folders = res.json() # парсим ответ

200


In [2]:
# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем ширину столбцов по контенту
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.options.mode.chained_assignment = None
# pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [3]:
# создаем словарь, где сохраним каждый продукт в отдельный датаФрейм со своим ключом
main_dict = {}

In [4]:
get_data_from_ya_folder(yandex_folders, main_dict, flag='prog')                     


/01_yandex
/02_prog
beeline_plan_игроник_ингосстрах_кис_лето_banners_020724
plan_игроник_рсхб_beeline_свой вклад_banners_160724 (1)
plan_ингосстрах_ beeline_хореографы_banners_240924 (3)
beeline_plan_igronik_бриф по тендерам_101024
/03_Полина


In [5]:
df = pd.concat(main_dict, ignore_index=True)
df = df.fillna('')

df['views'] = df['views'].astype('float').round(0)
int_columns = ['reach', 'impressions', 'clicks', 'views']
df[int_columns] = df[int_columns].apply(lambda x: x.astype('int'))

In [6]:
float_columns = ['unit price', 'budget_without_nds', 'budget_nds', 'vtr, %']
df[float_columns] = df[float_columns].apply(lambda x: x.astype('float')).round(2)

In [7]:
df

11,supplier,report_name,sheet_name,brand,period,source,site/ssp,placement,targeting,geo,soc_dem,ad copy format,rotation type,unit price,frequency,reach,impressions,clicks,budget_without_nds,budget_nds,views,"vtr, %"
0,beeline,beeline_plan_игроник_ингосстрах_кис_лето_banners_020724,Plan_Media,Ингосстрах_КИС_Лето,15.07-14.08.24,beeline AI,DSP,Desktop + mobile \n(+in-app),"ГЕО: РФ \nЦА: Все 18+\nПользователи, кто заходил на сайты/установлено приложение конкурентов в категории страхования (Росгосстрах, АльфаСтрахование, Тинькофф Страхование, СберСтрахование) и в банковской категории (Сбер, Альфа, Тинькофф, Россгосстрах, Согаз и т.д.)",рф,все 18+,Banners,CPM,88.0,3,7666666,23000000,29900,2024000.00,2428800.00,0,0.0
1,beeline,beeline_plan_игроник_ингосстрах_кис_лето_banners_020724,Plan_Media,Ингосстрах_КИС_Лето,15.07-14.08.24,beeline AI,DSP,Desktop + mobile \n(+in-app),ГЕО: РФ\nЦА: Все 18+\nКлиенты Сбера по транзакционным смс и установленному приложению,рф,все 18+,Banners,CPM,88.0,3,10000000,30000000,39000,2640000.00,3168000.00,0,0.0
2,beeline,plan_игроник_рсхб_beeline_свой вклад_banners_160724 (1),Plan_Media,РСХБ_Свой вклад,ноябрь-декабрь 24 г.,beeline AI,DSP,Desktop + mobile \n(+in-app),"ГЕО: РФ, кроме Крыма\nЦА: Все 25-65, ядро 30-45 (Ж 50%/М 50%)\n• Доход от 30 000 руб/мес, ядро от 70 000 руб/мес\n• Интерес к вкладам\n• Пользователи, которые интересуются банковскими продуктами, посещают разделы банковских порталов и сайтов о вкладах\n• Установлено приложение/посещают сайты/получают смс от банков-конкурентов\n• Интересуются вступлением в наследство, порядком наследования\n• Посетители, которых интересует купле-продажа недвижимости, земельных участков\n• Ключевые слова ""вклады","рф, кроме крыма","все 25-65, ядро 30-45 (ж 50%/м 50%)",Banners,CPM,88.0,3,15782828,47348485,61553,4166666.68,5000000.02,0,0.0
3,beeline,plan_ингосстрах_ beeline_хореографы_banners_240924 (3),Plan_Media,Ингосстрах,октябрь 2024 г.,beeline AI,DSP,Desktop + mobile \n(+in-app),"ГЕО: Казань, Уфа, Новосибирск, Екатеринбург, Астрахань, Нижний Новгород, Санкт-Петербург, Саратов\nЦА: Все 18 +\n• Посетители сайтов: цифровая-педагогика.рф, art-center.ru, dancerus.ru, balletcontest.ru, dancewind.ru, baltgem.ru, saluttalantov.ru, nvfest.ru, tourkids.ru\n• Интерес к курсам, посетители обучающих платформ для педагогов-хореографов\n• Интерес к одежде и обуви для танцев\n• KW: танцевальные/хореографические фестивали и конкурсы","казань, уфа, новосибирск, екатеринбург, астрахань, нижний новгород, санкт-петербург, саратов",все 18 +,Banners,CPM,88.0,3,1893939,5681819,7386,500000.07,600000.09,0,0.0
4,beeline,beeline_plan_igronik_бриф по тендерам_101024,Plan_Media,Биннофарм и Noname,1 месяц 2025 г.,beeline AI,DSP,Desktop + mobile \n(+in-app),"Портал о женском здоровье\nГЕО: РФ\nЦА: Ж 18+\n• Покупатели брендов конкурентов или категории препаратов (ОФД)\n• Аудитория, которая посещает сайты конкурентов\n• Аудитория с интересом к медицинским услугам по направлению ""Гинекология""\n• Посетители онлайн-аптек, которых интересуют гинекологические препараты\n• Участники женских интернет-сообществ, посетители журналов и порталов для женщин\n• Гинекологи (посещение профильных сайтов, приложений для врачей гинекологического профиля, интерес к курсам повышения квалификации, интерес к выставкам/форумам/конференциям для гинекологов) \n• KW: эндометриоз, пероральные контрацептивы, лечение анке, прогестероновая недостаточность, бесплодие, выкидыш, синдром предменструального напряжения, дисменорея, аменорея, дисфункциональные маточные кровотечения, инфекции половых органов/кожи/слизистых оболочек, ВПЧ, вагинит, кольпит",рф,ж 18+,Banners,CPM,98.9,3,5360000,16080000,20904,1590312.00,1908374.40,0,0.0
5,beeline,beeline_plan_igronik_бриф по тендерам_101024,Plan_Media,Биннофарм и Noname,1 месяц 2025 г.,beeline AI,DSP,Desktop + mobile \n(+in-app),"Портал о женском здоровье\nГЕО: РФ\nЦА: Ж 18+\n• Покупатели брендов конкурентов или категории препаратов (ОФД)\n• Аудитория, которая посещает сайты конкурент

In [ ]:
test = df[df['report_name']=='рсхб_игроник_ segmento _ рсхб_up_2024']

In [ ]:
test = df[df['sheet_name']=='mediaplan 1000к']

In [ ]:
test

In [ ]:
test.iloc[1]

In [ ]:
test[['reach', 'impressions', 'clicks', 'budget_without_nds']].sum()

In [ ]:
df

In [ ]:
# создаем подключение к Гуглу
client = create_connection(service_key)

In [ ]:
# прописываем путь и подключаемсяк нужному листу Гугл докса
sh = client.open_by_url(google_sheet_link)
sh.share(gmail, perm_type='user', role='writer')
google_sheet = sh.worksheet(sheet_name)

In [ ]:
# выгружем наш датаФрейм на лист
google_sheet.clear()
export_dataframe_to_google_sheet(google_sheet, final_report)

In [ ]:
main_dict['beeline_plan_игроник_ингосстрах_кис_лето_banners_020724']

In [ ]:
main_dict['plan_игроник_рсхб_beeline_свой вклад_banners_160724 (1)'].iloc[0]

In [ ]:
main_dict['plan_ингосстрах_ beeline_хореографы_banners_240924 (3)']

In [ ]:
main_dict['beeline_plan_igronik_бриф по тендерам_101024']